# Figure 3: Saturation magnitudes for SW NIRCam Filters w/ WLP8 weak lens

***
### Table of Contents

1. [Information](#Information)
2. [Imports](#Imports)
3. [Data](#Data)
4. [Generate the Saturation Magnitudes Plot](#Generate-the-Saturation-Magnitudes-Plot)
6. [Issues](#Issues)
7. [About this Notebook](#About-this-Notebook)
***

## Information

#### JDox links: 
* [NIRCam Bright Source Limits](https://jwst-docs.stsci.edu/display/JTI/NIRCam+Bright+Source+Limits#NIRCamBrightSourceLimits-Time-seriesimagingsaturationlimits)
    * Figure 3: Saturation magnitudes for short wavelength NIRCam filters using the WLP8 weak lens with a 160 × 160 pixel subarray

## Imports

In [ ]:
import os
from astropy.io import ascii
from astropy.table import Table
import requests
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
%matplotlib inline

## Data

#### Data Location: 

The data is stored in a NIRCam JDox Box folder here:
[ST-INS-NIRCAM -> JDox -> nircam_bright_source_limits](https://stsci.box.com/s/t6ssiswd6teqpfagr582w349hu1aitn9)

In [ ]:
files = [('https://stsci.box.com/shared/static/r7rh1n1047lmjgxpo2hf44978yna8ct1.txt', 'saturation64.txt')]

In [ ]:
def download_file(url, file_name, output_directory='./', overwrite=False):
    """Download a file from Box given the direct URL

    Parameters
    ----------
    url : str
        URL to the file to be downloaded
        
    file_name : str
        The name of the file being downloaded
        
    output_directory : str
        Directory to download file_name into
        
    overwrite : str
        If False and the file to download already exists, the download
        will be skipped. If True, the file will be downloaded regardless
        of whether it already exists in output_directory

    Returns
    -------
    download_filename : str
        Name of the downloaded file
    """
    download_filename = os.path.join(output_directory, file_name)
    if not os.path.isfile(download_filename) or overwrite is True:
        print("Downloading {}".format(file_name))
        with requests.get(url, stream=True) as response:
            if response.status_code != 200:
                raise RuntimeError("Wrong URL - {}".format(url))
            with open(download_filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=2048):
                    if chunk:
                        f.write(chunk)
    else:
        print("{} already exists. Skipping download.".format(download_filename))
    return download_filename

#### Load the data

(The next cell assumes you downloaded the data into your ```Users/$(logname)/``` home directory)

In [ ]:
if os.environ.get('LOGNAME') is None:
    raise ValueError("WARNING: LOGNAME environment variable not set!")

In [ ]:
box_directory = os.path.join("/Users/", os.environ['LOGNAME'], "box_data")   
box_directory

In [ ]:
if not os.path.isdir(box_directory):
    try:
        os.mkdir(box_directory)
    except:
        raise OSError("Unable to create {}".format(box_directory))

In [ ]:
for file_info in files:
    file_url, filename = file_info
    outfile = download_file(file_url, filename, output_directory=box_directory)

In [ ]:
txt = Table.read(os.path.join(box_directory, 'saturation64.txt'),header_start=4,format='ascii')
txt

## Generate the Saturation Magnitudes Plot

In [ ]:
#set up colors
spectral = cm = plt.get_cmap('nipy_spectral')
cNorm  = mpl.colors.Normalize(vmin=0.0, vmax=5.)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=spectral)

#set up minor tick marks
XminorLocator = MultipleLocator(0.1)
YminorLocator = MultipleLocator(0.1)

In [ ]:
allowedfilters = 'F140M F150W F182M F187N F200W F210M F212N'.split()  # Time Series Imaging

plt.figure(figsize=(10,6))

ss = []
cc = []
allcolors = []
for line in txt:
    filter = line['Filter']
    saturation = line['Saturation']
    center = line['Center']
    lambda1 = line['Lambda1']
    lambda2 = line['Lambda2']

    if filter not in allowedfilters:
        continue

    if 'W' in filter:
        sym = 'D'
        lw = 1
        color = 'w'
        ms = 12
        fontweight = 'normal'
        fontsize = 10
        color = 'k'
        style = 'normal'
    elif 'M' in filter:
        sym = 'o'
        lw = 3
        color = 'k'
        ms = 8
        fontweight = 'bold'
        fontsize = 8
        color = 'k'
        style = 'normal'
        #continue
    elif 'N' in filter:
        sym = '+'
        lw = 5
        color = 'k'
        ms = 8
        fontweight = 'normal'
        fontsize = 10
        color= 'k'
        style = 'italic'
        #continue

    dy = 0.9
    dy = 1.5
    saturation_full = float(saturation)
    saturation_160 = saturation_full - 4.06
    saturation = float(saturation_160)  # WLP8 160x160
    y = saturation
    dy = 0.23
    if filter in 'F115W F322W2 F470N F480M F200W F187N'.split():
        dy = -0.42
    plt.text(center, y-dy, filter, ha='center', fontsize=fontsize, fontweight=fontweight, color=color, style=style)
    plt.plot([lambda1, lambda2], [saturation, saturation], 'k', lw=lw, zorder=-10)
    ss.append(float(saturation))
    cc.append(float(center))
    midpt = cc
    colorVal = scalarMap.to_rgba(float(center)-0.2)
    allcolors.append(colorVal)

plt.scatter(cc,ss,s=250, color=allcolors,alpha=0.6)
plt.clim(0.3, 5.3)
plt.xlabel('Wavelength (microns)',fontsize=17)
plt.ylabel('Saturation K mag (Vega) for G2V star',fontsize=17)
plt.title('Saturation (80% well) WLP8 + 160x160 subarray (2 reads)',fontsize=18)
ml = MultipleLocator(5)
plt.axes().yaxis.set_minor_locator(ml)
plt.xlim(0.5,5.1)
plt.ylim(-0.5,6.5,1)
plt.clim(0.3, 5.3)
plt.gca().invert_yaxis()
plt.tick_params(labelsize=20)
plt.tick_params(axis='both', right='off', top='False')

## Issues

* None

## About this Notebook
**Authors:**   
Alicia Canipe & Dan Coe 

**Updated On:**   
April 05, 2019